In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

2024-02-01 17:15:19.216464: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Extrait les dernières données et écrit le dataset

In [2]:
from lib.dataeng import Etl

In [3]:
etl = Etl()

df_weight_raw, df_food_raw, df_exos_raw, df_all = etl.extract_store_all()

Le fichier poids le plus récent est : data_BEN_1706761334.zip, parmi :

data_BEN_1706593344.zip
data_BEN_1706676305.zip
data_BEN_1706761334.zip
Le fichier food le plus récent est : File-Export-2020-08-03-to-2024-02-01.zip, parmi 

File-Export-2020-08-03-to-2024-02-01.zip
File-Export-2020-08-03-to-2024-01-31.zip
File-Export-2020-08-03-to-2024-01-30.zip
Le fichier exercices le plus récent est : polar-user-data-export_703c11e2-d0c0-42d0-84a6-665ce3a9f847.zip, parmi :

polar-user-data-export_12baa4ce-79a5-40f8-ad40-74308f7c2c26.zip
polar-user-data-export_703c11e2-d0c0-42d0-84a6-665ce3a9f847.zip


100%|██████████| 1868/1868 [01:23<00:00, 22.45it/s]


In [13]:
fullds_path = '/home/benjamin/Folders_Python/Weight/data/full_dataset.csv'

with open(fullds_path, 'r') as f:
    df_full = pd.read_csv(f, index_col=0)
    
cols_to_drop = ['Masse_Osseuse', 'Masse_Musculaire', 'Masse_Hydrique']
df_full.drop(columns=cols_to_drop, inplace=True)
df_full.dropna(inplace=True)

In [16]:
df_full.describe(include='all').transpose()

,count,mean,std,min,25%,50%,75%,max
Masse_Totale,1254.0,73.738516,7.774613,66.000000,69.263542,71.350000,73.297009,107.250000
Masse_Grasse,1254.0,12.189070,5.160909,7.541667,8.934385,10.836500,12.139643,37.190000
MG%,1254.0,0.160785,0.043948,0.112927,0.128908,0.151511,0.166661,0.346760
BMR,1254.0,1699.468035,59.270358,1626.348571,1664.170750,1679.345200,1706.089600,1933.516000
Calories,1254.0,1968.121132,458.288696,0.300000,1780.625000,1850.650000,2039.700000,5762.000000
Lipides,1254.0,52.877751,22.354653,0.000000,38.625000,49.350000,64.000000,235.900000
Glucides,1254.0,213.171132,75.626152,0.100000,168.700000,199.200000,240.975000,875.400000
Proteines,1254.0,157.025199,39.911538,0.000000,133.900000,161.950000,184.325000,267.100000
exo_duree,1254.0,2696.149102,2800.143793,0.000000,0.000000,2326.293000,4753.248000,28800.000000
exo_cals_bruts,1254.0,295.417863,295.738840,0.000000,0.000000,275.500000,497.750000,2290.000000


In [106]:
ds = df_full[['Calories', 'exo_cals_nets', 'Masse_Totale', 'Masse_Grasse']]

In [107]:
ds

,Calories,exo_cals_nets,Masse_Totale,Masse_Grasse
2020-08-09,1861.3,0.000000,107.250000,37.190000
2020-08-11,1772.4,0.000000,104.650000,33.150000
2020-08-12,1894.3,0.000000,104.350000,33.800000
2020-08-13,1822.6,0.000000,104.400000,34.340000
2020-08-14,1891.5,0.000000,104.500000,33.850000
...,...,...,...,...
2024-01-27,1775.9,300.393829,73.350000,12.182857
2024-01-28,1821.9,294.128988,73.339167,12.444167
2024-01-29,1795.7,181.483944,73.896667,12.780000
2024-01-30,1680.9,0.000000,73.340000,12.521538


### Classe génératrice de fenêtres

On va créer des fenêtres :
- dans le passé, inputs (calories, exercices) seuls
- dans le futur, inputs (calories, exercices) + outputs (masse totale, masse grasse) pour prédiction

In [80]:
# from tensorflow time series tutorial
    
class WindowGenerator():
  def __init__(self, 
               past_length, future_length,
               dataset=None,
               label_columns=None):
    
    # Store the raw data.
    if dataset is not None:
      self.dataset = dataset
    else:
      self.dataset = ds

    # Work out the label column indices.

    if label_columns is not None:
      self.label_columns = label_columns
    else:
      self.label_columns = ['Masse_Totale', 'Masse_Grasse']
      
    self.label_columns_indices = {name: list(self.dataset.columns).index(name) for name in self.label_columns}
  
    self.column_indices = {name: i for i, name in enumerate(self.dataset.columns)}

    # Work out the window parameters.
    self.window_width = past_length + future_length
    self.label_width = future_length
    
    # Topology of window
    self.input_slice = slice(0, self.window_width)
    self.input_indices = np.arange(self.window_width)[self.input_slice]

    self.label_start = self.window_width - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.window_width)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.window_width}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',])

In [81]:
w1 = WindowGenerator(past_length=3, future_length=10)

w1

Total window size: 13
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
Label indices: [ 3  4  5  6  7  8  9 10 11 12]

In [87]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  
#   if self.label_columns is not None:
#     labels = tf.stack(
#         [labels[:, :, self.column_indices[name]] for name in self.label_columns],
#         axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.window_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [88]:
def make_dataset(self):
  data = np.array(self.dataset, dtype=np.float32)
  
  tds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.window_width,
      sequence_stride=1,
      # shuffle=True,
      batch_size=2,)

  tds = tds.map(self.split_window)

  return tds

WindowGenerator.make_dataset = make_dataset

In [89]:
w1 = WindowGenerator(past_length=3, future_length=10)

w1

Total window size: 13
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
Label indices: [ 3  4  5  6  7  8  9 10 11 12]

In [90]:
np.array(w1.dataset)

array([[1861.3       ,    0.        ,  107.25      ,   37.19      ],
       [1772.4       ,    0.        ,  104.65      ,   33.15      ],
       [1894.3       ,    0.        ,  104.35      ,   33.8       ],
       ...,
       [1795.7       ,  181.48394444,   73.89666667,   12.78      ],
       [1680.9       ,    0.        ,   73.34      ,   12.52153846],
       [1682.        ,  332.37208889,   73.57153846,   12.97307692]])

In [101]:
tds = tf.keras.utils.timeseries_dataset_from_array(
      data=np.array(w1.dataset),
      targets=None,
      sequence_length=w1.window_width,
      sequence_stride=1,
      # shuffle=True,
      batch_size=2,)

In [103]:
tds

<BatchDataset element_spec=TensorSpec(shape=(None, None, 4), dtype=tf.float64, name=None)>

In [104]:
tds = tds.map(w1.split_window)

In [105]:
tds

<MapDataset element_spec=(TensorSpec(shape=(None, 13, 4), dtype=tf.float64, name=None), TensorSpec(shape=(None, 10, 4), dtype=tf.float64, name=None))>

In [95]:
w1.label_columns_indices

{'Masse_Totale': 2, 'Masse_Grasse': 3}

In [55]:
tds = w1.make_dataset()

In [56]:
tds

<MapDataset element_spec=(TensorSpec(shape=(None, 13, 4), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10, 4), dtype=tf.float32, name=None))>

In [57]:
for e in tds.take(1):
    print(e)

(<tf.Tensor: shape=(2, 13, 4), dtype=float32, numpy=
array([[[1861.3     ,    0.      ,  107.25    ,   37.19    ],
        [1772.4     ,    0.      ,  104.65    ,   33.15    ],
        [1894.3     ,    0.      ,  104.35    ,   33.8     ],
        [1822.6     ,    0.      ,  104.4     ,   34.34    ],
        [1891.5     ,    0.      ,  104.5     ,   33.85    ],
        [1925.3     ,    0.      ,  104.25    ,   34.28    ],
        [2021.7     ,    0.      ,  104.05    ,   34.43    ],
        [1946.5     ,    0.      ,  103.9     ,   33.88    ],
        [1789.5     ,    0.      ,  103.55    ,   34.13    ],
        [1914.8     ,    0.      ,  103.25    ,   32.65    ],
        [1841.1     ,    0.      ,  103.45    ,   31.45    ],
        [1702.8     ,    0.      ,  103.35    ,   30.965   ],
        [1916.1     ,    0.      ,  103.2     ,   31.56    ]],

       [[1772.4     ,    0.      ,  104.65    ,   33.15    ],
        [1894.3     ,    0.      ,  104.35    ,   33.8     ],
        [1822.6